In [1]:
import pandas as pd
import numpy as np
import json
import re

In [2]:
df = pd.read_csv('Ashutosh.csv')

In [3]:
df.head()

,Title,JSON Response
0,Django,"{\n ""id"": 1250569,\n ""name"": ""Django"",\n..."
1,Coding: Intermediate level,"{\n ""id"": 1471468,\n ""name"": ""Coding: In..."
2,Docker,"{\n ""id"": 170977,\n ""name"": ""Docker"",\n ..."
3,ASP.NET,"{\n ""id"": 895134,\n ""name"": ""ASP.NET (in..."
4,Creating REST APIs,"{\n ""id"": 1950355,\n ""name"": ""Creating R..."


In [4]:
df.shape

(223, 2)

In [5]:
df.isnull().sum()

Title            4
JSON Response    5
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
df.shape

(217, 2)

In [8]:
df.columns

Index(['Title', 'JSON Response'], dtype='object')

In [9]:
df.isnull().sum()

Title            0
JSON Response    0
dtype: int64

In [10]:
df["JSON Response"].iloc[0]

'{\n    "id": 1250569,\n    "name": "Django",\n    "summary": "This Django test evaluates a candidate’s skills using the Django framework. This screening test will help you hire web developers and back-end engineers who can build reliable web applications with Django.",\n    "duration": 600.0,\n    "estimated_duration": null,\n    "type": [\n        {\n            "id": 7,\n            "name": "Programming skills",\n            "visible": true\n        }\n    ],\n    "is_new": false,\n    "content_type_name": "test_authoring.test",\n    "default_language": "en",\n    "description": "Django is a high-level Python web framework that encourages rapid development and clean, pragmatic design. Hiring candidates with strong Django skills can lead to faster development, improved software scalability, better security, cost savings, and greater flexibility.  \\n\\nThis test evaluates candidates’ abilities to use Django’s model system for effective database management, implement Django views, wri

In [11]:
df.rename(columns={df.columns[1]: 'json_response'}, inplace=True)

In [12]:
def is_valid_json(json_str):
    try:
        json.loads(json_str)
        return True
    except Exception as err:
        print("exception occured",err)
        return False

df = df[df['json_response'].apply(is_valid_json)].reset_index(drop=True)
df.shape

(217, 2)

In [13]:
df.shape

(217, 2)

In [14]:
df.isnull().sum()

Title            0
json_response    0
dtype: int64

In [15]:
def extract_type_name(json_str):
    try:
        data = json.loads(json_str)
        type_list = data.get("type", [])

        if isinstance(type_list, list) and type_list and isinstance(type_list[0], dict):
            return type_list[0].get("name") 
            
    except Exception as err:
        print("exception occured",err)

    return None 
        
df["type"] = df["json_response"].apply(extract_type_name)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Title          217 non-null    object
 1   json_response  217 non-null    object
 2   type           217 non-null    object
dtypes: object(3)
memory usage: 5.2+ KB


In [16]:
df.head()

,Title,json_response,type
0,Django,"{\n ""id"": 1250569,\n ""name"": ""Django"",\n...",Programming skills
1,Coding: Intermediate level,"{\n ""id"": 1471468,\n ""name"": ""Coding: In...",Programming skills
2,Docker,"{\n ""id"": 170977,\n ""name"": ""Docker"",\n ...",Software skills
3,ASP.NET,"{\n ""id"": 895134,\n ""name"": ""ASP.NET (in...",Programming skills
4,Creating REST APIs,"{\n ""id"": 1950355,\n ""name"": ""Creating R...",Programming skills


In [17]:
def extractJSON(json_str,key,subkey=None):
    try:
        parsed = json.loads(json_str)
        value = parsed.get(key,None)

        if isinstance(value,list) and len(value)>0 and isinstance(value[0],dict):
            return value[0].get(subkey,None) if subkey else value[0]
        return value
    except  Exception as err:
        print("exception occured",err)
        return None

In [18]:
df['questions'] = df['json_response'].apply(lambda x: extractJSON(x,'preview_questions','text'))

In [19]:
df.head()

,Title,json_response,type,questions
0,Django,"{\n ""id"": 1250569,\n ""name"": ""Django"",\n...",Programming skills,<p>You are working on a Django project for a c...
1,Coding: Intermediate level,"{\n ""id"": 1471468,\n ""name"": ""Coding: In...",Programming skills,"<p><span style=""background-color: transparent;..."
2,Docker,"{\n ""id"": 170977,\n ""name"": ""Docker"",\n ...",Software skills,"<p>Given the Dockerfile below, what is the bes..."
3,ASP.NET,"{\n ""id"": 895134,\n ""name"": ""ASP.NET (in...",Programming skills,<p>Which code snippet can you use to delay the...
4,Creating REST APIs,"{\n ""id"": 1950355,\n ""name"": ""Creating R...",Programming skills,"<p><span style=""background-color: transparent;..."


In [20]:
df['type'] = df['json_response'].apply(lambda x: extractJSON(x,'type','name'))

In [21]:
df.head()

,Title,json_response,type,questions
0,Django,"{\n ""id"": 1250569,\n ""name"": ""Django"",\n...",Programming skills,<p>You are working on a Django project for a c...
1,Coding: Intermediate level,"{\n ""id"": 1471468,\n ""name"": ""Coding: In...",Programming skills,"<p><span style=""background-color: transparent;..."
2,Docker,"{\n ""id"": 170977,\n ""name"": ""Docker"",\n ...",Software skills,"<p>Given the Dockerfile below, what is the bes..."
3,ASP.NET,"{\n ""id"": 895134,\n ""name"": ""ASP.NET (in...",Programming skills,<p>Which code snippet can you use to delay the...
4,Creating REST APIs,"{\n ""id"": 1950355,\n ""name"": ""Creating R...",Programming skills,"<p><span style=""background-color: transparent;..."


In [22]:
def cleanhtml(text):
    text = re.sub(r"<.*?>", '', text)
    text = text.replace('\n', ' ')
    text = text.replace('&nbsp;', ' ')
    text = text.strip()
    return text

df['questions'] = df['questions'].apply(cleanhtml)

In [23]:
df.head()

,Title,json_response,type,questions
0,Django,"{\n ""id"": 1250569,\n ""name"": ""Django"",\n...",Programming skills,You are working on a Django project for a comp...
1,Coding: Intermediate level,"{\n ""id"": 1471468,\n ""name"": ""Coding: In...",Programming skills,The MEX number of a non-negative set of number...
2,Docker,"{\n ""id"": 170977,\n ""name"": ""Docker"",\n ...",Software skills,"Given the Dockerfile below, what is the best w..."
3,ASP.NET,"{\n ""id"": 895134,\n ""name"": ""ASP.NET (in...",Programming skills,Which code snippet can you use to delay the lo...
4,Creating REST APIs,"{\n ""id"": 1950355,\n ""name"": ""Creating R...",Programming skills,You're building a high-throughput API for a cr...


In [24]:
df['question_type'] = df['json_response'].apply(lambda x:  extractJSON(x,'preview_questions','type'))

In [25]:
df.head()

,Title,json_response,type,questions,question_type
0,Django,"{\n ""id"": 1250569,\n ""name"": ""Django"",\n...",Programming skills,You are working on a Django project for a comp...,multiple-choice
1,Coding: Intermediate level,"{\n ""id"": 1471468,\n ""name"": ""Coding: In...",Programming skills,The MEX number of a non-negative set of number...,code
2,Docker,"{\n ""id"": 170977,\n ""name"": ""Docker"",\n ...",Software skills,"Given the Dockerfile below, what is the best w...",multiple-choice
3,ASP.NET,"{\n ""id"": 895134,\n ""name"": ""ASP.NET (in...",Programming skills,Which code snippet can you use to delay the lo...,multiple-choice
4,Creating REST APIs,"{\n ""id"": 1950355,\n ""name"": ""Creating R...",Programming skills,You're building a high-throughput API for a cr...,multiple-choice


In [26]:
df['question_type'].unique()

array(['multiple-choice', 'code', 'multiple-response', 'truefalse'],
      dtype=object)

In [27]:
df.isnull().sum()

Title            0
json_response    0
type             0
questions        0
question_type    0
dtype: int64

In [28]:
def map_question_type(q_type):
    mapping = {
        "multiple-choice": 2,
        "truefalse": 11,        
        "multiple-response": 15 
    }
    return mapping.get(q_type, None)
df["ques_type"] = df["question_type"].map(map_question_type)

In [29]:
df.drop(columns=["question_type"], inplace=True)

In [30]:
df = df.dropna(subset=["ques_type"])
df["ques_type"] = df["ques_type"].astype(int)

In [31]:
df.head()

,Title,json_response,type,questions,ques_type
0,Django,"{\n ""id"": 1250569,\n ""name"": ""Django"",\n...",Programming skills,You are working on a Django project for a comp...,2
2,Docker,"{\n ""id"": 170977,\n ""name"": ""Docker"",\n ...",Software skills,"Given the Dockerfile below, what is the best w...",2
3,ASP.NET,"{\n ""id"": 895134,\n ""name"": ""ASP.NET (in...",Programming skills,Which code snippet can you use to delay the lo...,2
4,Creating REST APIs,"{\n ""id"": 1950355,\n ""name"": ""Creating R...",Programming skills,You're building a high-throughput API for a cr...,2
5,Apex (Salesforce),"{\n ""id"": 1134919,\n ""name"": ""Apex (Sale...",Programming skills,Which one of the following trigger events shou...,2


In [32]:
print(df.groupby("ques_type").size())

ques_type
2     195
11      2
15      6
dtype: int64


In [33]:
df[df['questions'].apply(lambda x: bool(x))]

,Title,json_response,type,questions,ques_type
0,Django,"{\n ""id"": 1250569,\n ""name"": ""Django"",\n...",Programming skills,You are working on a Django project for a comp...,2
2,Docker,"{\n ""id"": 170977,\n ""name"": ""Docker"",\n ...",Software skills,"Given the Dockerfile below, what is the best w...",2
3,ASP.NET,"{\n ""id"": 895134,\n ""name"": ""ASP.NET (in...",Programming skills,Which code snippet can you use to delay the lo...,2
4,Creating REST APIs,"{\n ""id"": 1950355,\n ""name"": ""Creating R...",Programming skills,You're building a high-throughput API for a cr...,2
5,Apex (Salesforce),"{\n ""id"": 1134919,\n ""name"": ""Apex (Sale...",Programming skills,Which one of the following trigger events shou...,2
...,...,...,...,...,...
212,NetSuite Administration,"{\n ""id"": 1258902,\n ""name"": ""NetSuite A...",Software skills,"In a custom NetSuite sales order form, the ""Pa...",2
213,Advanced Networking in Amazon Web Services (AWS),"{\n ""id"": 1431343,\n ""name"": ""Advanced N...",Programming skills,As a solutions architect for a company operati...,2
214,Linux,"{\n ""id"": 925544,\n ""name"": ""Linux"",\n ...",Software skills,You are trying to access a Domain Name System ...,2
215,Wireshark,"{\n ""id"": 996346,\n ""name"": ""Wireshark"",...",Software skills,You are a system administrator in a finance co...,2


In [34]:
print(df.index)

Index([  0,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       207, 208, 209, 210, 211, 212, 213, 214, 215, 216],
      dtype='int64', length=203)


In [35]:
df = df.reset_index(drop=True)

In [36]:
def extract_options_and_score(row):
    json_data = json.loads(row['json_response'])
    preview_questions = json_data.get("preview_questions", [])
    
    options = []
    score = []

    if preview_questions:
        for question in preview_questions:
            options = [answer.get("text", "") for answer in question.get("answers", [])]
            score = [answer.get("text", "") for answer in question.get("answers", []) if answer.get("score", 0) > 0]
    
    return options, score
df[['options', 'score']] = df.apply(lambda row: pd.Series(extract_options_and_score(row)), axis=1)
df[['questions', 'options', 'score']].head()


,questions,options,score
0,You are working on a Django project for a comp...,[<p>Impersonate the guest authors and change t...,[<p>Apply the <code><strong>```title```</stron...
1,"Given the Dockerfile below, what is the best w...",[<p>The registry will only store the first ima...,[<p>The registry will only store the first ima...
2,Which code snippet can you use to delay the lo...,"[<pre class=""ql-syntax"" spellcheck=""false"">pri...","[<pre class=""ql-syntax"" spellcheck=""false"">pri..."
3,You're building a high-throughput API for a cr...,"[<pre class=""ql-syntax"" spellcheck=""false"">{\n...","[<pre class=""ql-syntax"" spellcheck=""false"">{\n..."
4,Which one of the following trigger events shou...,[<p>The Decorator pattern and the Strategy pat...,[<p>The Decorator pattern and the Strategy pat...


In [37]:
df['options'] = df['options'].apply(lambda options: [cleanhtml(opt) for opt in options] if isinstance(options, list) else options)
df['score'] = df['score'].apply(lambda answers: [cleanhtml(ans) for ans in answers] if isinstance(answers, list) else answers)
df.head()

,Title,json_response,type,questions,ques_type,options,score
0,Django,"{\n ""id"": 1250569,\n ""name"": ""Django"",\n...",Programming skills,You are working on a Django project for a comp...,2,[Impersonate the guest authors and change the ...,"[Apply the ```title``` HTML template filter, e..."
1,Docker,"{\n ""id"": 170977,\n ""name"": ""Docker"",\n ...",Software skills,"Given the Dockerfile below, what is the best w...",2,[The registry will only store the first image ...,[The registry will only store the first image ...
2,ASP.NET,"{\n ""id"": 895134,\n ""name"": ""ASP.NET (in...",Programming skills,Which code snippet can you use to delay the lo...,2,[private async Task BroadcastMarket() { ...,[private async Task BroadcastMarket() {...
3,Creating REST APIs,"{\n ""id"": 1950355,\n ""name"": ""Creating R...",Programming skills,You're building a high-throughput API for a cr...,2,"[{ “phone”: “123-123-1234” }, { “phone”: {phon...",[{ “phone”: “123-123-1234” }]
4,Apex (Salesforce),"{\n ""id"": 1134919,\n ""name"": ""Apex (Sale...",Programming skills,Which one of the following trigger events shou...,2,[The Decorator pattern and the Strategy patter...,[The Decorator pattern and the Strategy pattern]


In [38]:
df.isnull().sum()

Title            0
json_response    0
type             0
questions        0
ques_type        0
options          0
score            0
dtype: int64

In [39]:
df.rename(columns={'questions': 'title'}, inplace=True)

In [40]:
df.head()

,Title,json_response,type,title,ques_type,options,score
0,Django,"{\n ""id"": 1250569,\n ""name"": ""Django"",\n...",Programming skills,You are working on a Django project for a comp...,2,[Impersonate the guest authors and change the ...,"[Apply the ```title``` HTML template filter, e..."
1,Docker,"{\n ""id"": 170977,\n ""name"": ""Docker"",\n ...",Software skills,"Given the Dockerfile below, what is the best w...",2,[The registry will only store the first image ...,[The registry will only store the first image ...
2,ASP.NET,"{\n ""id"": 895134,\n ""name"": ""ASP.NET (in...",Programming skills,Which code snippet can you use to delay the lo...,2,[private async Task BroadcastMarket() { ...,[private async Task BroadcastMarket() {...
3,Creating REST APIs,"{\n ""id"": 1950355,\n ""name"": ""Creating R...",Programming skills,You're building a high-throughput API for a cr...,2,"[{ “phone”: “123-123-1234” }, { “phone”: {phon...",[{ “phone”: “123-123-1234” }]
4,Apex (Salesforce),"{\n ""id"": 1134919,\n ""name"": ""Apex (Sale...",Programming skills,Which one of the following trigger events shou...,2,[The Decorator pattern and the Strategy patter...,[The Decorator pattern and the Strategy pattern]


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Title          203 non-null    object
 1   json_response  203 non-null    object
 2   type           203 non-null    object
 3   title          203 non-null    object
 4   ques_type      203 non-null    int64 
 5   options        203 non-null    object
 6   score          203 non-null    object
dtypes: int64(1), object(6)
memory usage: 11.2+ KB


###### duplicate = df[df.duplicated(subset=["title"], keep=False)].shape[0]
duplicate

In [42]:
df.drop_duplicates(subset=["title"], inplace=True)
df.reset_index(drop=True, inplace=True)

In [43]:
df[['Title', 'title', 'ques_type', 'options', 'score']]

,Title,title,ques_type,options,score
0,Django,You are working on a Django project for a comp...,2,[Impersonate the guest authors and change the ...,"[Apply the ```title``` HTML template filter, e..."
1,Docker,"Given the Dockerfile below, what is the best w...",2,[The registry will only store the first image ...,[The registry will only store the first image ...
2,ASP.NET,Which code snippet can you use to delay the lo...,2,[private async Task BroadcastMarket() { ...,[private async Task BroadcastMarket() {...
3,Creating REST APIs,You're building a high-throughput API for a cr...,2,"[{ “phone”: “123-123-1234” }, { “phone”: {phon...",[{ “phone”: “123-123-1234” }]
4,Apex (Salesforce),Which one of the following trigger events shou...,2,[The Decorator pattern and the Strategy patter...,[The Decorator pattern and the Strategy pattern]
...,...,...,...,...,...
163,NetSuite Administration,"In a custom NetSuite sales order form, the ""Pa...",2,[The customer's browser version is outdated an...,[The credit card type is not supported by NetS...
164,Advanced Networking in Amazon Web Services (AWS),As a solutions architect for a company operati...,2,"[Amazon CloudFront, AWS Direct Connect, AWS Gl...",[Amazon CloudFront]
165,Linux,You are trying to access a Domain Name System ...,2,"[True, False]",[False]
166,Wireshark,You are a system administrator in a finance co...,2,"[sip.response-time, sip.response-time = 1, TCP...",[sip.response-time]


In [44]:
def format_to_template(row):
    return {
        "title": row['title'],
        "ques_type": row['ques_type'],
        "options": row['options'],
        "score": row['score']
    }


formatted_data = df.apply(format_to_template, axis=1).tolist()
formatted_df = pd.DataFrame(formatted_data)
formatted_df.to_excel('questions_kit.xlsx', index=False)